## Secant Method

__MATH 420__ <br>
_Spring 2021_ <br>
Section 2.4 <br>


Here is simple code for the secant method. We default the tolerance `tol` to $64$ times the machine epsilon of the initial guess $x_0$ and the maximum number of iterations `maxiter` to 1000. Both `tol` and `maxiter` are keyword arguments.

We'll stop when $f(x_k)$ is sufficiently samll.  For small, we'll use `!isapprox(f1,0, atol = tol, rtol=0)`.

We'll default the second initial guess for the zero to $x_0 + f(x_0)$. There isn't much mathematical logic behind this choice. But when $f(x_0)$ is small (and presumably $x_0$ is close to the true zero), we have $x_1$ close to $x_0$. It seems reasonable to choose $x_1$ close to $x_0$ when  $f(x_0)$ is small.

Each time through the loop, we compute only one new value for the function.

In [ ]:
function secant_method(f::Function,x0::Number,x1::Number = x0 + f(x0); tol = 64*eps(typeof(x0)), maxiter::Integer = 1000)
    f0 = f(x0)
    f1 = f(x1) 
    while !isapprox(f1,0.0, atol = tol, rtol=0) && !isnan(x1) && maxiter > 0 # early stop when x1 = NaN
        maxiter -= 1
        xx = x1 - f1 * (x1 - x0) / (f1 - f0) 
        (x0,x1,f0,f1) = (x1,xx,f1,f(xx)) 
    end    
    # error when too many iterations; otherwise, return the midpoint
    if maxiter <= 0
        error("secant_method: Exceeded maximum number of iterations.")    
    else     
       x1
    end
end;

Let's look for zeros of the function $x \mapsto x \exp(x) - 107$.  First, a graphical study--it looks like there is a zero that is very close to one.

In [ ]:
F = x -> x*exp(x) - 107

In [ ]:
using Gadfly

In [ ]:
plot(F,0,5)

There is a zero between $3$ and $4$. Let's try several values for the two initial values of the secant sequence.

In [ ]:
 secant_method(F, 0.0, 3.0)

In [ ]:
 secant_method(F, -1.5, 2.0)

In [ ]:
 secant_method(F, 1.2, 1.003, maxiter=10^4)

In [ ]:
 secant_method(F, 1.2, 1.003, maxiter=10^6)

In [ ]:
function secant_sequence(f::Function,x0::Number,x1::Number = x0 + f(x0); tol = 64*eps(typeof(x0)), maxiter::Integer = 1000)
    f0 = f(x0)
    f1 = f(x1) 
    pts = []
    push!(pts, x0)
    push!(pts, x1)
    while !isapprox(f1,0.0, atol = tol, rtol=0) && !isnan(x1) && maxiter > 0 # early stop when x1 = NaN
        maxiter -= 1
        xx = x1 - f1 * (x1 - x0) / (f1 - f0) 
        (x0,x1,f0,f1) = (x1,xx,f1,f(xx)) 
        push!(pts,x1)
    end    
    # error when too many iterations; otherwise, return the midpoint
    if maxiter <= 0
        error("secant_method: Exceeded maximum number of iterations.")    
    else     
       pts
    end
end;

In [ ]:
xx = secant_sequence(F, 2.0, 18.0)

In [ ]:
n = length(xx)

In [ ]:
xpts = [k for k = 1 : n];

The graph wobbles a bit, then approaches its asymptote fairly quickly:

In [ ]:
plot(x=xpts, y=xx)

In [ ]:
xf = last(xx)

In [ ]:
e = [xx[k] - xf for k = 1 : n];

In [ ]:
ee = [e[k+2]/(e[k] * e[k+1]) for k = 1 : n-2]

In [ ]:
plot(x=xpts[1:n-2], y=ee)

Let's check the value of $f^{\prime\prime} (z) /(2 f^\prime (z))$ where $z =  3.437952688840655 $. We could do this calculation by hand, but computing derivatives is easy using Julia's `ForwardDiff` package.

In [ ]:
using ForwardDiff

In [ ]:
dF = x -> ForwardDiff.derivative(F, x)

In [ ]:
ddF = x -> ForwardDiff.derivative(dF, x)

In [ ]:
xf

In [ ]:
(ddF(xf)/(2 * dF(xf)))


Looking at the values of the array `ee,` the values match the theoretical value farily well.

Bigfloat inputs are no problem--the same code works for `BigFloat` numbers. Here is a case the convergence is so fast there are identical consecutive terms. That gives a division by zero error and returns a `NaN`. We should modify the stopping condition to eliminate such division by zero errors.

In [ ]:
xx = secant_sequence(F, BigFloat("3.0"), BigFloat("4.0"))

In [ ]:
xpts = [k for k = 1 : length(xx)];

In [ ]:
plot(x=xpts, y=xx)

A trcky case: The true zero is 2, but the function of is very close to 0 in a largish neighborhood of 2.

In [ ]:
G = x -> (x-2)^42

In [ ]:
plot(G,1,3)

In [ ]:
plot(G,1.5,2.5)

In [ ]:
xxx = secant_method(G, 1.5, 2.2)

In [ ]:
xxx = secant_method(G, 1.5, 2.5, tol=1.0e-13)

This works, more-or-less, but even with a tolerance of $10^{-100}$ we get a value for the zero that differs from the true value by about $0.005$

In [ ]:
xxx = secant_method(G, 1.5, 3.0, tol=1.0e-100)

What's the story?  For the function $F = x \mapsto (x-2)^{42}$, we have  $F(2) = 0 $ and $F^\prime(2) = 0$. Because of this, we say that $2$ is a _degenerate zero_ of $F$. Our result that $E(k+2) = q E(k) E(k+1)$ is approximately correct for large $k$ makes the assumption that the zero isn't degenerate.
